<div style="text-align:center; background-color:blue; padding:10px;">
  <h1 style="color:white;">Premier chargement en base de données rds</h1>
</div>


<div style="text-align:left; background-color:gray; padding:10px;">
  <h1 style="color:white;">Import des librairies et des fichiers annexes</h1>
</div>


In [4]:
# Librairies pour la décompression de fichiers
import gzip
from io import BytesIO

# Librairies pour le scrapping
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Visualisation de données
import pandas as pd

# Connection à RDS
from connection import db,cursor,s3,bucket_name

# datas processing
import datas_processing

<div style="text-align:left; background-color:gray; padding:10px;">
  <h1 style="color:white;">Création des tables dans la base de données</h1>
</div>


In [15]:
query="""
CREATE DATABASE IF NOT EXISTS datagouv;
"""
cursor.execute(query)

query="""
USE datagouv;
"""
cursor.execute(query)

# Lecture de fichier create_tables
with open('create_tables.sql') as file:
    sql_queries = file.read()
try :
    for query in sql_queries.split(';')[:-1]:
        query = query.strip()
        cursor.execute(query)
except Exception as e:
    print(f"Erreur dans la creation des tables : {e}")
    db.rollback()

query="""
SHOW TABLES;
"""
cursor.execute(query)
tables = cursor.fetchall()

# Afficher les tables
for table in tables:
    print(f"La table {table[0]} à bien été créée")

La table COMMUNES à bien été créée
La table DEPARTEMENTS à bien été créée
La table REGIONS à bien été créée
La table TYPES_BIENS à bien été créée
La table VENTES à bien été créée


<div style="text-align:left; background-color:gray; padding:10px;">
  <h1 style="color:white;">Récupération des datas sur :<br>
  L'api https://geo.api.gouv.fr/ <br>
  et Scrapping des datas sur :
  https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres-geolocalisees/</h1>
</div>


#### - Les régions depuis l'api

In [17]:
url_regions = "https://geo.api.gouv.fr/regions"
response_regions = requests.get(url_regions)
data_regions = response_regions.json()
df_regions = pd.DataFrame(data_regions)
df_regions = df_regions.loc[:,["code","nom"]]
df_regions = df_regions.drop_duplicates()
df_regions.columns = ['ID_REGION','Name_region']
df_regions.head()

,ID_REGION,Name_region
0,11,Île-de-France
1,24,Centre-Val de Loire
2,27,Bourgogne-Franche-Comté
3,28,Normandie
4,32,Hauts-de-France


In [18]:
# Insertion dans la table REGIONS
query="""
USE datagouv;
"""
cursor.execute(query)

df_regions['ID_REGION'] = df_regions.ID_REGION.astype(str)
df_regions['Name_region'] = df_regions.Name_region.astype(str)

for index, row in df_regions.iterrows():
    query="""
    INSERT IGNORE INTO REGIONS (ID_REGION,Name_region)
    VALUES (%s,%s)
    """
    cursor.execute(query,(row["ID_REGION"],row["Name_region"]))
db.commit()

query="""
SELECT * FROM REGIONS
"""
cursor.execute(query)
regions = cursor.fetchall()

# Afficher les tables
for region in regions:
    print(f"ID_region: {region[0]} -- Nom region : {region[1]}")

ID_region: 01 -- Nom region : Guadeloupe
ID_region: 02 -- Nom region : Martinique
ID_region: 03 -- Nom region : Guyane
ID_region: 04 -- Nom region : La Réunion
ID_region: 06 -- Nom region : Mayotte
ID_region: 11 -- Nom region : Île-de-France
ID_region: 24 -- Nom region : Centre-Val de Loire
ID_region: 27 -- Nom region : Bourgogne-Franche-Comté
ID_region: 28 -- Nom region : Normandie
ID_region: 32 -- Nom region : Hauts-de-France
ID_region: 44 -- Nom region : Grand Est
ID_region: 52 -- Nom region : Pays de la Loire
ID_region: 53 -- Nom region : Bretagne
ID_region: 75 -- Nom region : Nouvelle-Aquitaine
ID_region: 76 -- Nom region : Occitanie
ID_region: 84 -- Nom region : Auvergne-Rhône-Alpes
ID_region: 93 -- Nom region : Provence-Alpes-Côte d'Azur
ID_region: 94 -- Nom region : Corse


#### - Les départements depuis l'api

In [19]:
# Récupération des codes departements, noms departements et code régions
url = "https://geo.api.gouv.fr/departements"
response = requests.get(url)
data_dpts = response.json()
df_depts = pd.DataFrame(data_dpts)
df_depts = df_depts.loc[:,["code","nom","codeRegion"]]
df_depts = df_depts.drop_duplicates()
df_depts.columns = ['ID_DEPT', 'Name_departement', 'ID_REGION']
df_depts.head()

,ID_DEPT,Name_departement,ID_REGION
0,01,Ain,84
1,02,Aisne,32
2,03,Allier,84
3,04,Alpes-de-Haute-Provence,93
4,05,Hautes-Alpes,93


In [20]:
# Insertion dans la table DEPARTEMENTS
for index, row in df_depts.iterrows():
    query="""
    INSERT IGNORE INTO DEPARTEMENTS (ID_DEPT, Name_departement,	ID_REGION)
    VALUES (%s,%s,%s)
    """
    cursor.execute(query,(row["ID_DEPT"],row["Name_departement"],row["ID_REGION"]))
db.commit()

query="""
SELECT * FROM DEPARTEMENTS
LIMIT 10
"""
cursor.execute(query)
dapartements = cursor.fetchall()

# Afficher les tables
for departement in dapartements:
    print(f"Id departement: {departement[0]} -- Nom departement : {departement[1]} -- Id region : {departement[2]}")

Id departement: 01 -- Nom departement : Ain -- Id region : 84
Id departement: 02 -- Nom departement : Aisne -- Id region : 32
Id departement: 03 -- Nom departement : Allier -- Id region : 84
Id departement: 04 -- Nom departement : Alpes-de-Haute-Provence -- Id region : 93
Id departement: 05 -- Nom departement : Hautes-Alpes -- Id region : 93
Id departement: 06 -- Nom departement : Alpes-Maritimes -- Id region : 93
Id departement: 07 -- Nom departement : Ardèche -- Id region : 84
Id departement: 08 -- Nom departement : Ardennes -- Id region : 44
Id departement: 09 -- Nom departement : Ariège -- Id region : 76
Id departement: 10 -- Nom departement : Aube -- Id region : 44


#### - Les communes depuis l'api

In [21]:
# Récupération des codes communes , noms communes et code départements 
url = "https://geo.api.gouv.fr/communes"
response = requests.get(url)
data_communes = response.json()
df_communes = pd.DataFrame(data_communes)
df_communes = df_communes.loc[:,["code","nom","codeDepartement"]]
df_communes = df_communes.drop_duplicates()
df_communes.columns= ["ID_COMMUNE","NAME_COMMUNE","ID_DEPT"]
df_communes.head()

,ID_COMMUNE,NAME_COMMUNE,ID_DEPT
0,01001,L'Abergement-Clémenciat,01
1,01002,L'Abergement-de-Varey,01
2,01004,Ambérieu-en-Bugey,01
3,01005,Ambérieux-en-Dombes,01
4,01006,Ambléon,01


In [22]:
# Insertion dans la table COMMUNES
for index, row in df_communes.iterrows():
    query="""
    INSERT IGNORE INTO COMMUNES (ID_COMMUNE, NAME_COMMUNE,	ID_DEPT)
    VALUES (%s,%s,%s)
    """
    cursor.execute(query,(row["ID_COMMUNE"],row["NAME_COMMUNE"],row["ID_DEPT"]))
db.commit()

query="""
SELECT * FROM COMMUNES
LIMIT 10
"""
cursor.execute(query)
communes = cursor.fetchall()

# Afficher les tables
for commune in communes:
    print(f"Id commune: {commune[0]} -- Nom commune : {commune[1]} -- Id départment : {commune[2]}")

Id commune: 01001 -- Nom commune : L'Abergement-Clémenciat -- Id départment : 01
Id commune: 01002 -- Nom commune : L'Abergement-de-Varey -- Id départment : 01
Id commune: 01004 -- Nom commune : Ambérieu-en-Bugey -- Id départment : 01
Id commune: 01005 -- Nom commune : Ambérieux-en-Dombes -- Id départment : 01
Id commune: 01006 -- Nom commune : Ambléon -- Id départment : 01
Id commune: 01007 -- Nom commune : Ambronay -- Id départment : 01
Id commune: 01008 -- Nom commune : Ambutrix -- Id départment : 01
Id commune: 01009 -- Nom commune : Andert-et-Condon -- Id départment : 01
Id commune: 01010 -- Nom commune : Anglefort -- Id départment : 01
Id commune: 01011 -- Nom commune : Apremont -- Id départment : 01


In [6]:
# Insertion dans la table TYPES_BIENS
liste_type_local = ["Appartement","Maison"]
for local in liste_type_local:
    query="""
    INSERT IGNORE INTO TYPES_BIENS (NAME_TYPE_BIEN)
    VALUES(%s)
    """
    cursor.execute(query,(local))
db.commit()

query="""
SELECT * FROM TYPES_BIENS
"""
cursor.execute(query)
types_biens = cursor.fetchall()

# Affichage du contenu de la table TYPES_BIENS
for i in types_biens:
    print(f"Id type bien : {i[0]} -- nom : {i[1]}")

Id type bien : 1 -- nom : Appartement
Id type bien : 2 -- nom : Maison


#### - Les ventes en webscrapping depuis https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres-geolocalisees/

In [8]:
# URL de la page web
url = 'https://files.data.gouv.fr/geo-dvf/latest/csv/'

response = requests.get(url)
html_content = response.content

# Utilisez BeautifulSoup pour analyser le HTML
soup = BeautifulSoup(html_content, 'html.parser')

In [9]:
# Trouver toutes les balises <a>
a_tags = soup.find_all('a')
annees=[]
# Parcourir chaque balise <a> et extraire le texte ainsi que la date/heure
for a_tag in a_tags:
    name = a_tag.text.strip()
    if name!="../":
        annees.append(name)
print(annees)

['2018/', '2019/', '2020/', '2021/', '2022/', '2023/']


In [11]:
# La dernière année ne sera pas prise en compte pour tester la mise à jour des 
# données dans le programme maj_datas.py
for annee in annees[:-1] : 
    url = f'https://files.data.gouv.fr/geo-dvf/latest/csv/{annee}'
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Trouver la balise 'a' qui contient les liens de téléchargement
    csv_element = soup.find('pre').find('a', {'href': "full.csv.gz"})
    csv_link = urljoin(url, csv_element['href'])

    # Téléchargement des datas et convertion en dataframe
    csv_response = requests.get(csv_link)
    if csv_response.status_code == 200:
        # Utilisation du buffer pour décompresser le fichier .gz
        try :
            with BytesIO(csv_response.content) as file_buffer:
                with gzip.GzipFile(fileobj=file_buffer, mode='rb') as gz_file:
                    df = pd.read_csv(gz_file, low_memory=False)
        except Exception as e:
            print("Erreur dans la transformation des données en dataframe :",e)

    else :
        print("Problème dans le lien de récupération de données !!!")

    # Selection des données
    datas_processing.select_datas(df)
       
    # Gestion des valeurs manquantes :
    datas_processing.nan_management(df)

    # Adaptation des données du dataframe au format de la base de données
    datas_processing.format_data(df)

    #Suppression des lignes dupliquées
    df = df.drop_duplicates()

    # Chargement en base de données
    try :
        for index,row in df.iterrows():
            query="""
            INSERT IGNORE INTO VENTES (DATE_MUTATION, MONTANT,
                            NUMERO_RUE, RUE , CODE_POSTAL,ID_COMMUNE,
                            ID_TYPE_BIEN, NB_PIECES ,
                            SURFACE_BATI,  SURFACE_TERRAIN,
                            LONGITUDE, LATITUDE )
            VALUES (%s,%s,
                    %s,%s,%s,%s,
                    (SELECT ID_TYPE_BIEN FROM TYPES_BIENS WHERE NAME_TYPE_BIEN=%s),%s,
                    %s,%s,
                    %s,%s)
            """
            cursor.execute(query,(row["date_mutation"],row['valeur_fonciere'],
                        row["adresse_numero"],row["adresse_nom_voie"],row["code_postal"],row["code_commune"],
                        row["type_local"],row["nombre_pieces_principales"],
                        row["surface_reelle_bati"],row["surface_terrain"],
                        row["longitude"],row["latitude"]))
        db.commit()
        print(f" Les ventes de l'année {annee} ont bien été insérées dans RDS")
    except Exception as e:
        print(f"Erreur dans le chargement des données de l'année {annee} dans RDS :",e)
    finally:
        cursor.close()
        db.close()

KeyError: "['code_communetype_local'] not in index"

<div style="text-align:left; background-color:gray; padding:10px;">
  <h3 style="color:white;">Récupération de la dernière date de mise à jour des données et stockage dans le bucket S3</h3>
</div>


In [7]:
date_element = soup.find('pre').contents[-1].strip()
date_maj = date_element.replace(" ","")
print(date_maj)

# Utilisation de BytesIO pour créer un buffer en mémoire
buffer = BytesIO()
buffer.write(date_maj.encode('utf-8'))  # Encode la chaîne en UTF-8 avant de l'écrire

# Écriture du texte dans le fichier sur S3
buffer.seek(0)  # Réinitialise la position de lecture/écriture
s3.upload_fileobj(buffer, bucket_name, 'app_immo/last_maj.txt')

# Assurez-vous de fermer le buffer BytesIO
buffer.close()

13-Oct-202320:28-


In [8]:
# Téléchargement du fichier depuis S3 vers un nouveau buffer
buffer_read = BytesIO()
s3.download_fileobj(bucket_name, 'app_immo/last_maj.txt', buffer_read)

# Lecture du contenu du buffer
buffer_read.seek(0)
last_maj_saved = buffer_read.read().decode('utf-8')  # Decodez les bytes en UTF-8
print(last_maj_saved)

13-Oct-202320:28-


In [10]:
# ajout d'une valeur quelconque pour tester la lamba 
start_value="start_value"

# Utilisation de BytesIO pour créer un buffer en mémoire
buffer = BytesIO()
buffer.write(start_value.encode('utf-8'))  # Encode la chaîne en UTF-8 avant de l'écrire

# Écriture du texte dans le fichier sur S3
buffer.seek(0)  # Réinitialise la position de lecture/écriture
s3.upload_fileobj(buffer, bucket_name, 'app_immo/last_maj.txt')

# Assurez-vous de fermer le buffer BytesIO
buffer.close()